In [ ]:
import ROOT
import numpy as np
import pandas as pd
import copy

In [ ]:
tileboards = pd.read_csv("tileboards_layout_v10.csv")
tileboards["boundaries"  ] = tileboards["boundaries"  ].apply(lambda x: np.array(x.split(","), dtype=np.float64))
tileboards["n_rings"     ] = tileboards["n_rings"     ].apply(lambda x: np.array(x.split(","), dtype=np.int64  ))
tileboards["tb_names"    ] = tileboards["tb_names"    ].apply(lambda x: np.array(x.split(",")                  ))
tileboards["tb_areas_cm2"] = tileboards["tb_areas_cm2"].apply(lambda x: np.array(x.split(","), dtype=np.float64))

In [ ]:
tileboard_layout = []

for i in range(9,23):
    bounds       = tileboards["boundaries"  ][i - 9]
    n_rings      = tileboards["n_rings"     ][i - 9]
    tb_names     = tileboards["tb_names"    ][i - 9]
    tb_areas_cm2 = tileboards["tb_areas_cm2"][i - 9]
    
    for r in range(len(bounds) - 1):
        tp = ROOT.TPaveText(i - 0.5, bounds[r], i + 0.5, bounds[r + 1])
        
        if n_rings[r] != 1:
            tp_text = tp.AddText("%s%s" % (tb_names[r], n_rings[r]))
            tp_text.SetTextFont(42)
            tp_text.SetTextSize(0.02)
        
        tileboard_layout.append(tp.Clone())

In [ ]:
def calc_area(tbs=tileboards, name="", in_boards={"D8": 5}):
    
    names = ["A6", "B12", "C5", "D8", "E8", "F8", "F7", "F6", "F4", "G3", "G5", "G6", "G7", "G8"]
    
    n_boards = {i: 0 for i in names}
    n_boards.update(in_boards)
    area_boards = {}
    n_sipms = {i: int(i[1:]) for i in names}

    for i in range(9,23):
        bounds       = tbs["boundaries"  ][i - 9]
        n_rings      = tbs["n_rings"     ][i - 9]
        tb_names     = tbs["tb_names"    ][i - 9]
        tb_areas_cm2 = tbs["tb_areas_cm2"][i - 9]
        
        for r in range(len(bounds) - 1):
            if n_rings[r] != 1:
                board = "%s%s" % (tb_names[r], n_rings[r])
                area  = tb_areas_cm2[r]
                area  = round(area/10000, 2)
                area_boards[board] = area
    
    print("For %s \\\\" % name)

    total_area = 0
    for key in area_boards:
        total_area += n_boards[key] * area_boards[key]
    
    total_area = round(total_area, 2)

    print(*["%s - %s" % (i, n_boards[i]) for i in n_boards if n_boards[i] != 0], sep=", ", end=" \\\\ \n")
    
    total_sipms = 0
    for key in n_sipms:
        total_sipms += n_boards[key] * n_sipms[key] 

    print("Total SiPMs = %s, %s \\\\" % (288 * total_sipms, 2 * 288 * total_sipms))
    print("Scint. Area in one CE-H = %s $m^2$ \\\\" % total_area)
    print("Total Scint. Area = %s $m^2$ \\\\" % float(2 * total_area))
    print("... \\\\")
    
    return 288 * total_sipms, total_area

In [ ]:
reduced_set = pd.DataFrame()

csv_dir = "csv_output"

scints = [
    (24, "Molded (Individual)"),
    (38, "Cast (Array)"       ),
    (48, "Cast (Individual)"  ),
]

sipms = [
    ("18.37_40.0", "HE"      ),
    ("14.4_40.0" , "HDR2-10um"),
    ("34.5_40.0" , "HDR2-15um"),
]

#sipms area
areas = [
    2.0,
    4.0,
]

mip_values         = []
scint_type         = []
pde_ratio_values   = []
sipm_device        = [] 
sipm_area_values   = []
layer_values       = []
cell_area_values   = []
SN_ratio_values    = []
r_inner_values     = []
r_outer_values     = []

tbox_tiles = []
tbox_board = []

for scint in scints:
    
    for sipm in sipms:
        
        for area in areas:
            
            inputFileName = f"{csv_dir}/mip_{scint[0]}_pde_ratio_{sipm[0]}_sipm_area_{area}.csv"

            df = pd.read_csv(inputFileName, index_col=0)
            
            for layer in range(9,23):
                
                n_rings = tileboards["n_rings"][layer - 9]
                
                bin_r_inner = df.loc[df["layer"] == layer]["inner"      ].values
                bin_r_outer = df.loc[df["layer"] == layer]["outer"      ].values
                bin_value   = df.loc[df["layer"] == layer]["S/N"        ].values
                s_area      = df.loc[df["layer"] == layer]["sipm_area"  ].values
                c_area      = df.loc[df["layer"] == layer]["area (cm2)" ].values

                first = True
                r_inners = []
                r_outers = []
                tbox     = []
                tbox_f   = []

                start_ring = 0
                for i in range(len(n_rings)):
                    
                    subset_r_in  = []
                    subset_r_out = []
                    
                    end_ring = start_ring + n_rings[i]
                    
                    for ring in range(start_ring, end_ring):
                        r_in  = bin_r_inner[ring]
                        r_out = bin_r_outer[ring]
                        value = bin_value  [ring]
                        s_a   = s_area     [ring]
                        c_a   = c_area     [ring]

                        if first and value >= 5.0:
                            mip_values         .append(scint[0]     )
                            scint_type         .append(scint[1]     )
                            pde_ratio_values   .append(sipm[0]      )
                            sipm_device        .append(sipm[1]      )
                            sipm_area_values   .append(s_a          )
                            layer_values       .append(layer        )
                            cell_area_values   .append(c_a          )
                            SN_ratio_values    .append(value        )
                            first = False

                        if value >= 5.0:
                            r_inners.append(r_in )
                            r_outers.append(r_out)
                            subset_r_in.append(r_in)
                            subset_r_out.append(r_out)
                            tbox.append(ROOT.TBox(layer - 0.5, r_in, layer + 0.5, r_out))

                    if len(subset_r_in) == n_rings[i]:
                        r1 = min(subset_r_in)
                        r2 = max(subset_r_out)
                        tbox_f.append(ROOT.TBox(layer - 0.5, r1, layer + 0.5, r2))
                
                    start_ring = end_ring
                            
                if len(r_inners) != 0:
                    r_inner_values.append(r_inners)
                    r_outer_values.append(r_outers)
                    tbox_tiles.append(tbox)
                    tbox_board.append(tbox_f)
                            

reduced_set["mip"         ] = mip_values
reduced_set["scint_type"  ] = scint_type
reduced_set["pde_ratio"   ] = pde_ratio_values
reduced_set["sipm_device" ] = sipm_device
reduced_set["sipm_area"   ] = sipm_area_values
reduced_set["layer"       ] = layer_values
reduced_set["cell_area"   ] = cell_area_values
reduced_set["SN_ratio"    ] = SN_ratio_values
reduced_set["r_inners"    ] = r_inner_values 
reduced_set["r_outers"    ] = r_outer_values 

reduced_set["tbox_tiles"] = tbox_tiles
reduced_set["tbox_board"] = tbox_board

In [ ]:
def plot_and_save(d_set, device_maps, partial_devices="", t_layout=tileboard_layout,
                  legend_header="Scene", legY1=0.5, filename="plots/scene"):
    
    # --------------------------
    # read full t_boards to draw
    # -------------------------
    sipm_area_   = []
    scint_type_  = []
    sipm_device_ = []
    leg_texts    = []
    fill_colors  = []
    fill_styles  = []
    
    for device_map in device_maps:
        sipm_area_  .append(device_map["sipm_area"  ])
        scint_type_ .append(device_map["scint_type" ])
        sipm_device_.append(device_map["sipm_device"])
        leg_texts   .append(device_map["leg_t"      ])
        fill_colors .append(device_map["f_color"    ])
        fill_styles .append(device_map["f_style"    ])  

    p_sipm_area_   = []
    p_scint_type_  = []
    p_sipm_device_ = []
    p_leg_texts    = []
    p_fill_colors  = []
    p_fill_styles  = []
    
    for p_device in partial_devices:
        p_sipm_area_  .append(p_device["sipm_area"  ])
        p_scint_type_ .append(p_device["scint_type" ])
        p_sipm_device_.append(p_device["sipm_device"])
        p_leg_texts   .append(p_device["leg_t"      ])
        p_fill_colors .append(p_device["f_color"    ])
        p_fill_styles .append(p_device["f_style"    ])  

    canvas = ROOT.TCanvas("", "", 800, 600)
    canvas.SetRightMargin(0.3)

    detector_frame = canvas.DrawFrame(8, 850, 23, 2720,  "; CEH - Layer; R (mm)")
    detector_frame.Draw()

    legend = ROOT.TLegend(0.71, legY1, 0.97, 0.9)
    legend.SetTextSize(0.025)
    legend.SetBorderSize(0)

    for tbox in t_layout:
        tbox.SetFillColor(ROOT.kRed)
        tbox.Draw("l")

    legend.AddEntry(tbox, "S/N < 5", "f")
    legend.SetHeader(legend_header)

    # -----------------------------
    # draw partial t_boards
    # -----------------------------
    for a, b, c, d, e, f in zip(p_sipm_area_, p_scint_type_, p_sipm_device_, p_leg_texts, p_fill_colors, p_fill_styles):
        
        sub_d_set = d_set.loc[(d_set["sipm_area"] == a) & (d_set["scint_type"] == b) & (d_set["sipm_device"] == c)]
        
        for tboxes in sub_d_set["tbox_tiles"].values:
            for tbox in tboxes:
                tbox.SetFillColor(e)
                tbox.SetFillStyle(f)
                tbox.Draw()
    
    # -----------------------------
    # draw full t_boards
    # -----------------------------
    
    for a, b, c, d, e, f in zip(sipm_area_, scint_type_, sipm_device_, leg_texts, fill_colors, fill_styles):

        sub_d_set = d_set.loc[(d_set["sipm_area"] == a) & (d_set["scint_type"] == b) & (d_set["sipm_device"] == c)]
        
        for tboxes in sub_d_set["tbox_board"].values:
            for tbox in tboxes:
                tbox.SetFillColor(e)
                tbox.SetFillStyle(f)
                tbox.Draw()                

        if d == "":
            tt = "#splitline{Scint. - %s}{Device - %s (%1.0f mm^{2})}" % (b, c.replace("um", "#mum"), a)
            legend.AddEntry(tboxes[0], tt, "f")
        else:
            legend.AddEntry(tboxes[0], d, "f")
            
    # -----------------------------
    # -----------------------------
    
    cloned_tileboards = copy.deepcopy(t_layout)
    for tbox in cloned_tileboards:
        tbox.SetFillStyle(0)
        tbox.Draw("l")

    legend.Draw()
    canvas.Draw()
    canvas.Print(filename+".pdf")
    #canvas.Print(filename+".png")
    
    return canvas.Clone()

In [ ]:
hdr15_cs_4 = {"sipm_area": 4.0, "scint_type": "Cast (Individual)"  , "sipm_device": "HDR2-15um", "f_color": ROOT.kYellow + 1, "f_style": 1001, "leg_t": ""}
hdr15_cs_2 = {"sipm_area": 2.0, "scint_type": "Cast (Individual)"  , "sipm_device": "HDR2-15um", "f_color": ROOT.kYellow    , "f_style": 1001, "leg_t": ""}
hdr15_ms_4 = {"sipm_area": 4.0, "scint_type": "Molded (Individual)", "sipm_device": "HDR2-15um", "f_color": ROOT.kBlue   + 2, "f_style": 1001, "leg_t": ""}
hdr15_ms_2 = {"sipm_area": 2.0, "scint_type": "Molded (Individual)", "sipm_device": "HDR2-15um", "f_color": ROOT.kBlue      , "f_style": 1001, "leg_t": ""}
hdr15_ca_4 = {"sipm_area": 4.0, "scint_type": "Cast (Array)"   , "sipm_device": "HDR2-15um", "f_color": ROOT.kGreen  + 2, "f_style": 1001, "leg_t": ""}
hdr15_ca_2 = {"sipm_area": 2.0, "scint_type": "Cast (Array)"   , "sipm_device": "HDR2-15um", "f_color": ROOT.kGreen     , "f_style": 1001, "leg_t": ""}

hdr10_cs_4 = {"sipm_area": 4.0, "scint_type": "Cast (Individual)"  , "sipm_device": "HDR2-10um", "f_color": ROOT.kYellow + 1, "f_style": 1001, "leg_t": ""}
hdr10_cs_2 = {"sipm_area": 2.0, "scint_type": "Cast (Individual)"  , "sipm_device": "HDR2-10um", "f_color": ROOT.kYellow    , "f_style": 1001, "leg_t": ""}
hdr10_ms_4 = {"sipm_area": 4.0, "scint_type": "Molded (Individual)", "sipm_device": "HDR2-10um", "f_color": ROOT.kBlue   + 2, "f_style": 1001, "leg_t": ""}
hdr10_ms_2 = {"sipm_area": 2.0, "scint_type": "Molded (Individual)", "sipm_device": "HDR2-10um", "f_color": ROOT.kBlue      , "f_style": 1001, "leg_t": ""}
hdr10_ca_4 = {"sipm_area": 4.0, "scint_type": "Cast (Array)"   , "sipm_device": "HDR2-10um", "f_color": ROOT.kGreen  + 2, "f_style": 1001, "leg_t": ""}
hdr10_ca_2 = {"sipm_area": 2.0, "scint_type": "Cast (Array)"   , "sipm_device": "HDR2-10um", "f_color": ROOT.kGreen     , "f_style": 1001, "leg_t": ""}

he_cs_4 = {"sipm_area": 4.0, "scint_type": "Cast (Individual)"  , "sipm_device": "HE", "f_color": ROOT.kYellow + 1, "f_style": 1001, "leg_t": ""}
he_cs_2 = {"sipm_area": 2.0, "scint_type": "Cast (Individual)"  , "sipm_device": "HE", "f_color": ROOT.kYellow    , "f_style": 1001, "leg_t": ""}
he_ms_4 = {"sipm_area": 4.0, "scint_type": "Molded (Individual)", "sipm_device": "HE", "f_color": ROOT.kBlue   + 2, "f_style": 1001, "leg_t": ""}
he_ms_2 = {"sipm_area": 2.0, "scint_type": "Molded (Individual)", "sipm_device": "HE", "f_color": ROOT.kBlue      , "f_style": 1001, "leg_t": ""}
he_ca_4 = {"sipm_area": 4.0, "scint_type": "Cast (Array)"   , "sipm_device": "HE", "f_color": ROOT.kGreen  + 2, "f_style": 1001, "leg_t": ""}
he_ca_2 = {"sipm_area": 2.0, "scint_type": "Cast (Array)"   , "sipm_device": "HE", "f_color": ROOT.kGreen     , "f_style": 1001, "leg_t": ""}

In [ ]:
plot_dir = "plot_output"

c1 = plot_and_save(reduced_set, [hdr15_cs_2, hdr15_ca_2, hdr15_ms_2], [hdr15_cs_2],
                   legend_header="HDR2-15#mum", legY1=0.4, filename=f"{plot_dir}/HDR-3015_2mm")

c2 = plot_and_save(reduced_set, [hdr15_cs_4, hdr15_ca_4, hdr15_ms_4], [hdr15_cs_4],
                   legend_header="HDR-15#mum", legY1=0.4, filename=f"{plot_dir}/HDR-3015_4mm")

c3 = plot_and_save(reduced_set, [hdr10_cs_2, hdr10_ca_2, hdr10_ms_2], 
                   legend_header="HDR2-10#mum", legY1=0.4, filename=f"{plot_dir}/HDR-3010_2mm")

c4 = plot_and_save(reduced_set, [hdr10_cs_4, hdr10_ca_4, hdr10_ms_4], 
                   legend_header="HDR2-10#mum", legY1=0.4, filename=f"{plot_dir}/HDR-3010_4mm")

c5 = plot_and_save(reduced_set, [he_cs_2, he_ca_2, he_ms_2], 
                   legend_header="HE", legY1=0.4, filename=f"{plot_dir}/HE_2mm")

c6 = plot_and_save(reduced_set, [he_cs_4, he_ca_4, he_ms_4], 
                   legend_header="HE", legY1=0.4, filename=f"{plot_dir}/HE_4mm")

In [ ]:
c1.Draw()

In [ ]:
c2.Draw()

In [ ]:
c3.Draw()

In [ ]:
c4.Draw()

In [ ]:
c5.Draw()

In [ ]:
c6.Draw()

In [ ]:
hdr15_cs_2_var1 = copy.deepcopy(hdr15_cs_2)
hdr15_cs_2_var1["f_style"] = 3001

hdr15_ca_4_var1 = copy.deepcopy(hdr15_ca_4)
hdr15_ca_4_var1["f_style"] = 3344

hdr15_ms_4_var1 = copy.deepcopy(hdr15_ms_4)
hdr15_ms_4_var1["f_style"] = 3244

c1 = plot_and_save(reduced_set, [hdr15_cs_4, hdr15_cs_2, hdr15_ms_4, hdr15_ms_2], [hdr15_cs_4],
                   legend_header="Scene A", legY1=0.3, filename=f"{plot_dir}/HDR-3015_scene_A")
c2 = plot_and_save(reduced_set, [hdr15_cs_4, hdr15_ms_4, hdr15_cs_2, hdr15_ms_2], [hdr15_cs_4],
                   legend_header="Scene B", legY1=0.3, filename=f"{plot_dir}/HDR-3015_scene_B")
c3 = plot_and_save(reduced_set, [hdr15_cs_4, hdr15_cs_2, hdr15_ca_4, hdr15_ca_2, hdr15_ms_2], [hdr15_cs_4],
                   legend_header="Scene C", legY1=0.2, filename=f"{plot_dir}/HDR-3015_scene_C")
c3_var = plot_and_save(reduced_set, [hdr15_cs_4, hdr15_cs_2, hdr15_ca_4_var1, hdr15_ca_2, hdr15_ms_2], [hdr15_cs_4],
                   legend_header="Scene C", legY1=0.2, filename=f"{plot_dir}/HDR-3015_scene_C_1")

c4 = plot_and_save(reduced_set, [hdr15_cs_4, hdr15_cs_2, hdr15_ca_4, hdr15_ca_2, hdr15_ms_4, hdr15_ms_2], [hdr15_cs_4],
                   legend_header="Scene D", legY1=0.15, filename=f"{plot_dir}/HDR-3015_scene_D")
c4_var = plot_and_save(reduced_set, [hdr15_cs_4, hdr15_cs_2, hdr15_ca_4_var1, hdr15_ca_2, hdr15_ms_4, hdr15_ms_2], [hdr15_cs_4],
                   legend_header="Scene D", legY1=0.15, filename=f"{plot_dir}/HDR-3015_scene_D_1")

In [ ]:
c1.Draw();
s2_1, m1 = calc_area(name="Molded $2mm^2$", in_boards={"D8": 3, "E8": 5, "G6": 1, "G8": 5})
s4_1, m2 = calc_area(name="Molded $4mm^2$", in_boards={"D8": 3, "E8": 3, "G7": 1, "G8": 1})
s2_2, ci1 = calc_area(name="Cast-I $2mm^2$", in_boards={"A6": 1, "B12": 5, "D8": 3, "E8": 2, "G3": 1, "G5": 1})
s4_2, ci2 = calc_area(name="Cast-I $4mm^2$", in_boards={"A6": 3, "B12": 3, "C5": 2, "D8": 5, "F6": 1, "F4": 1, "F7": 1, "F8": 1})
print(m1 + m2, 2*(m1 + m2), round((m1 + m2)*100/184.61, 2), "\\%")
print(ci1 + ci2, 2*(ci1 + ci2), round((ci1 + ci2)*100/184.61, 2), "\\%")
print(s2_1 + s2_2, 2*(s2_1 + s2_2))
print(s4_1 + s4_2, 2*(s4_1 + s4_2))

In [ ]:
c2.Draw();
s2_1, m1 = calc_area(name="Molded $2mm^2$", in_boards={"D8": 3, "E8": 5, "G6": 1, "G8": 5})
s2_2, ci1 = calc_area(name="Cast-I $2mm^2$", in_boards={"A6": 1, "B12": 5, "D8": 6, "E8": 5, "G3": 1, "G5": 1, "G7": 1, "G8": 1})
s4_1, ci2 = calc_area(name="Cast-I $4mm^2$", in_boards={"A6": 3, "B12": 3, "C5": 2, "D8": 5, "F4": 1, "F6": 1, "F7": 1, "F8": 1})
print(m1, 2*(m1), round((m1)*100/184.61, 2), "\\%")
print(ci1 + ci2, 2*(ci1 + ci2), round((ci1 + ci2)*100/184.61, 2), "\\%")
print(s2_1 + s2_2, 2*(s2_1 + s2_2))
print(s4_1, 2*(s4_1))

In [ ]:
c3.Draw()
s2_1, m1 = calc_area(name="Molded $2mm^2$", in_boards={"D8": 3, "E8": 5, "G6": 1, "G8": 5})
s2_2, ca1 = calc_area(name="Cast-A $2mm^2$", in_boards={"B12": 1, "D8": 4, "E8": 3, "G5": 1, "G7": 1, "G8": 1})
s4_1, ca2 = calc_area(name="Cast-A $4mm^2$", in_boards={"A6": 3, "B12": 4, "D8": 2, "E8": 2, "F8": 1, "G3": 1})
s4_2, ci1 = calc_area(name="Cast-I $4mm^2$", in_boards={"A6": 1, "B12": 3, "C5": 2, "D8": 5, "F4": 1, "F6": 1, "F7": 1})
print(m1, 2*(m1), round((m1)*100/184.61, 2), "\\%")
print(ci1, 2*(ci1), round((ci1)*100/184.61, 2), "\\%")
print(ca1 + ca2, 2*(ca1 + ca2), round((ca1 + ca2)*100/184.61, 2), "\\%")
print(s2_1 + s2_2, 2*(s2_1 + s2_2))
print(s4_1 + s4_2, 2*(s4_1 + s4_2))

In [ ]:
c3_var.Draw()

In [ ]:
c4.Draw()
s2_1, m1 = calc_area(name="Molded $2mm^2$", in_boards={"D8": 3, "E8": 5, "G6": 1, "G8": 5})
s4_1, m2 = calc_area(name="Molded $4mm^2$", in_boards={"D8": 3, "E8": 3, "G7": 1, "G8": 1})
s2_2, ca1 = calc_area(name="Cast-A $2mm^2$", in_boards={"B12": 1, "D8": 1, "G5": 1})
s4_2, ca2 = calc_area(name="Cast-A $4mm^2$", in_boards={"A6": 3, "B12": 4, "D8": 2, "E8": 2, "F8": 1, "G3": 1})
s4_3, ci1 = calc_area(name="Cast-I $4mm^2$", in_boards={"A6": 1, "B12": 3, "C5": 2, "D8": 5, "F4": 1, "F6": 1, "F7": 1})
print(m1 + m2, 2*(m1 + m2), round((m1 + m2)*100/184.61, 2), "\\%")
print(ci1, 2*(ci1), round((ci1)*100/184.61, 2), "\\%")
print(ca1 + ca2, 2*(ca1 + ca2), round((ca1 + ca2)*100/184.61, 2), "\\%")
print(s2_1 + s2_2, 2*(s2_1 + s2_2))
print(s4_1 + s4_2 + s4_3, 2*(s4_1 + s4_2 + s4_3))

In [ ]:
c4_var.Draw()